* 영화 이름을 가지고 감독, 주요 출연진, 예산, 흥행 수익, 영화의 장르, 간단한 시놉시스 등 영화에 대한 정보로 답장하는 체인을 만드세요.

* LLM은 항상 동일한 형식을 사용하여 응답해야 하며, 이를 위해서는 원하는 출력의 예시를 LLM에 제공해야 합니다.

* 예제를 제공하려면 FewShotPromptTemplate 또는 FewShotChatMessagePromptTemplate을 사용하세요.

In [10]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.globals import set_llm_cache, set_debug
from langchain.cache import SQLiteCache
from langchain.callbacks import get_openai_callback

set_llm_cache(SQLiteCache("db/movie_cache.db"))
set_debug(True)

chat = ChatOpenAI(
    temperature=0.1,
    # streaming=True,
    # callbacks=[
    #     StreamingStdOutCallbackHandler()
    # ],
)

examples = [
    {
        "question": "Tell me about the movie 'The Dark Knight'",
        "answer": """
            The movie 'The Dark Knight' is a action movie.
            It was directed by Christopher Nolan.
            The main actors are Christian Bale, Heath Ledger, Aaron Eckhart.
            The budget was $185 million.
            The revenue was $1.004 billion.
            The genre is action.
            The synopsis is "When the menace known as the Joker emerges from his mysterious past, he wreaks havoc and chaos on the people of Gotham. The Dark Knight must accept one of the greatest psychological and physical tests of his ability to fight injustice."
        """
    },
    {
        "question": "Tell me about the movie 'Shrek'",
        "answer": """
            The movie 'Shrek' is a comedy movie.
            It was directed by Andrew Adamson, Vicky Jenson.
            The main actors are Mike Myers, Eddie Murphy, Cameron Diaz.
            The budget was $125 million.
            The revenue was $484 million.
            The genre is comedy.
            The synopsis is "A mean lord exiles fairytale character Shrek to the swamp of a neighboring castle, where he must find true love to break the spell."
        """
    },
    {
        "question": "Tell me about the movie 'The Lord of the Rings'",
        "answer": """
            The movie 'The Lord of the Rings' is a fantasy movie.
            It was directed by Peter Jackson.
            The main actors are Elijah Wood, Ian McKellen, Viggo Mortensen.
            The budget was $281 million.
            The revenue was $2.917 billion.
            The genre is fantasy.
            The synopsis is "A meek Hobbit from the Shire and eight companions set out on a journey to destroy the powerful One Ring and save Middle-earth from the Dark Lord Sauron."
        """
    }
]

example_prompt = ChatPromptTemplate.from_messages([
    ("human", "What is the movie {question}?"),
    ("ai", "{answer}")
])

example_prompt = FewShotChatMessagePromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
)

final_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant that can answer questions about movies."),
    example_prompt,
    ("human", "What is the movie {question}?"),
])

chain = final_prompt | chat

with get_openai_callback() as usage:
    a = chain.invoke({"question": "Avatar"})
    print(a)
    print(usage)

[chain/start] [1:chain:RunnableSequence] Entering Chain run with input:
{
  "question": "Avatar"
}
[chain/start] [1:chain:RunnableSequence > 2:prompt:ChatPromptTemplate] Entering Prompt run with input:
{
  "question": "Avatar"
}
[chain/end] [1:chain:RunnableSequence > 2:prompt:ChatPromptTemplate] s] Exiting Prompt run with output:
{
  "lc": 1,
  "type": "constructor",
  "id": [
    "langchain",
    "prompts",
    "chat",
    "ChatPromptValue"
  ],
  "kwargs": {
    "messages": [
      {
        "lc": 1,
        "type": "constructor",
        "id": [
          "langchain",
          "schema",
          "messages",
          "SystemMessage"
        ],
        "kwargs": {
          "content": "You are a helpful assistant that can answer questions about movies.",
          "additional_kwargs": {}
        }
      },
      {
        "lc": 1,
        "type": "constructor",
        "id": [
          "langchain",
          "schema",
          "messages",
          "HumanMessage"
        ],
    